In [22]:
import pandas as pd
import pandas as pd
import numpy as np
import cvxpy as cp
import yfinance as yf
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [39]:
# Load the stock data.
file_path = '.\data\s&p500\clean_s&p500_stock_data.csv'
stock_data = pd.read_csv(file_path, index_col='date', parse_dates=True)

In [40]:
stock_data.head()

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,1268.800049,23.962805,37.450001,43.520000,2.669643,NaN,9.44,NaN,18.976089,6.120000,...,18.570000,58.470001,27.014999,39.288540,NaN,16.804457,66.485435,42.830002,76.480003,NaN
2006-01-04,1273.459961,24.027182,39.200001,43.830002,2.677500,NaN,9.62,NaN,19.004877,6.151111,...,18.660000,58.570000,27.264999,39.235836,NaN,16.948238,67.116508,42.410000,77.019997,NaN
2006-01-05,1273.479980,24.656652,39.709999,44.040001,2.656429,NaN,9.55,NaN,19.249577,6.061111,...,18.650000,58.279999,26.955000,38.814228,NaN,17.670740,66.407768,42.529999,77.720001,NaN
2006-01-06,1285.449951,24.785408,39.000000,43.990002,2.725000,NaN,9.75,NaN,19.619022,6.173333,...,18.719999,59.430000,27.375000,38.577076,NaN,17.620417,66.067963,44.119999,78.529999,NaN
2006-01-09,1290.150024,24.713877,38.610001,44.560001,2.716071,NaN,10.15,NaN,20.348318,6.116667,...,18.670000,59.400002,27.645000,39.104084,NaN,17.757010,68.407768,44.790001,77.879997,NaN


In [41]:
stock_data.tail(5)

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2022-12-23,3844.820068,149.229996,12.71,143.279999,131.860001,163.100006,381.019989,NaN,108.180000,63.380001,...,70.930000,108.680000,31.830000,14.61,109.730003,128.899994,126.690002,248.220001,48.450001,145.759995
2022-12-27,3829.250000,149.550003,12.53,145.020004,130.029999,162.990005,381.019989,NaN,108.570000,63.619999,...,71.570000,110.190002,32.070000,14.70,110.720001,129.899994,127.279999,251.000000,48.840000,145.300003
2022-12-28,3783.219971,148.089996,12.32,145.300003,126.040001,162.229996,381.019989,NaN,107.830002,62.599998,...,70.570000,108.379997,30.980000,14.37,108.940002,129.309998,125.989998,246.839996,47.970001,143.830002
2022-12-29,3849.280029,151.089996,12.70,146.309998,129.610001,162.559998,381.019989,NaN,110.309998,63.110001,...,71.070000,109.199997,32.279999,14.49,111.639999,129.990005,127.830002,257.529999,49.080002,148.149994
2022-12-30,3839.500000,149.649994,12.72,147.029999,129.929993,161.610001,381.019989,NaN,109.790001,62.779999,...,70.110001,110.300003,31.840000,14.60,110.570000,128.080002,127.500000,256.410004,49.160000,146.550003


In [42]:
# Calculate daily returns
stock_returns = stock_data.pct_change().dropna(how='all')
stock_returns.tail()

,^GSPC,A,AAL,AAP,AAPL,ABBV,ABMD,ABS,ABT,ACGL,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2022-12-23,0.005868,0.001476,0.011943,0.008446,-0.002798,-0.001041,0.0,0.0,0.001389,0.008433,...,0.012852,0.026445,0.011118,-0.003411,-0.000728,0.000621,-0.000789,0.002869,0.003521,0.005033
2022-12-27,-0.004050,0.002144,-0.014162,0.012144,-0.013878,-0.000674,0.0,0.0,0.003605,0.003787,...,0.009023,0.013894,0.007540,0.006160,0.009022,0.007758,0.004657,0.011200,0.008050,-0.003156
2022-12-28,-0.012021,-0.009763,-0.016760,0.001931,-0.030685,-0.004663,0.0,0.0,-0.006816,-0.016033,...,-0.013972,-0.016426,-0.033988,-0.022449,-0.016077,-0.004542,-0.010135,-0.016574,-0.017813,-0.010117
2022-12-29,0.017461,0.020258,0.030844,0.006951,0.028324,0.002034,0.0,0.0,0.022999,0.008147,...,0.007085,0.007566,0.041963,0.008351,0.024784,0.005259,0.014604,0.043307,0.023139,0.030035
2022-12-30,-0.002541,-0.009531,0.001575,0.004921,0.002469,-0.005844,0.0,0.0,-0.004714,-0.005229,...,-0.013508,0.010073,-0.013631,0.007591,-0.009584,-0.014693,-0.002582,-0.004349,0.001630,-0.010800


In [31]:
stock_returns.isna().sum().sum()

330147


# Experiment setup
- daily return을 기준으로 index tracking 설정
- 월별 리벨런싱 가정
- 2006/01/01 ~ 2022-12/30
- k act: 10% 20% 30% 40% 50% - 전체 주식의 % 

# Algorithm

## CARD

In [17]:
def card(X, y, kact):
    n, p = X.shape
    beta = cp.Variable(p)
    z = cp.Variable(p, boolean=True)
    
    objective = cp.Minimize((1/n) * cp.sum_squares(y - X @ beta))
    constraints = [cp.sum(beta) == 1, beta >= 0, cp.sum(z) <= kact, beta <= z]
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.GUROBI)
    
    return beta.value

## ALASSO Algorithm

In [18]:
def alasso(X, y, kact, max_iter=100, tol=1e-4, nu=1):
    n, p = X.shape
    beta = cp.Variable(p)
    weights = np.ones(p)
    
    for _ in range(max_iter):
        objective = cp.Minimize((1/n) * cp.sum_squares(y - X @ beta) + cp.norm1(cp.multiply(weights, beta)))
        constraints = [cp.sum(beta) == 1, beta >= 0]
        problem = cp.Problem(objective, constraints)
        problem.solve()
        
        beta_value = beta.value
        new_weights = 1 / (np.abs(beta_value) + tol)
        
        if np.linalg.norm(new_weights - weights) < tol:
            break
        weights = new_weights
    
    return beta.value


## Sorted LASSO

In [ ]:
def slope_slc(X, y, kact, lambdas):
    n, p = X.shape
    beta = cp.Variable(p)
    
    # Create the sorted L1 norm penalty
    def sorted_l1_norm(beta, lambdas):
        abs_beta = cp.abs(beta)
        sorted_beta = cp.Variable(p)
        constraints = [sorted_beta == cp.mixed_norm(abs_beta, 1, axis=0)]
        return lambdas @ sorted_beta
    
    objective = cp.Minimize((1/n) * cp.sum_squares(y - X @ beta) + sorted_l1_norm(beta, lambdas))
    constraints = [cp.sum(beta) == 1, beta >= 0]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return beta.value


## MSW_LASSO

In [16]:
def msw_lasso(X, y, kact, max_iter=100, tol=1e-4, penalty='MCP', a=3.7):
    n, p = X.shape
    beta = cp.Variable(p)
    weights = np.ones(p)
    
    # Define penalty functions
    if penalty == 'MCP':
        def p_lambda(beta_j, lam, a):
            return lam * beta_j - (beta_j**2) / (2 * a) if beta_j <= a * lam else (a * lam**2) / 2
    elif penalty == 'SCAD':
        def p_lambda(beta_j, lam, a):
            if beta_j <= lam:
                return lam * beta_j
            elif beta_j <= a * lam:
                return (-beta_j**2 + 2 * a * lam * beta_j - lam**2) / (2 * (a - 1))
            else:
                return (a + 1) * lam**2 / 2
    # Add other penalties if needed

    for _ in range(max_iter):
        objective = cp.Minimize((1/n) * cp.sum_squares(y - X @ beta) + cp.norm1(cp.multiply(weights, beta)))
        constraints = [cp.sum(beta) == 1, beta >= 0]
        problem = cp.Problem(objective, constraints)
        problem.solve()
        
        beta_value = beta.value
        new_weights = np.array([abs(p_lambda(beta_value[j], 1, a)) for j in range(p)])
        
        if np.linalg.norm(new_weights - weights) < tol:
            break
        weights = new_weights
    
    return beta.value

# Backtest
## Experiment result 
- Turn over 비용
- Tracking error(for out of sample)
- Computing time(running time second)

In [19]:
def evaluate_portfolio(X, y, beta):
    tracking_error = np.sqrt(np.mean((y - X @ beta)**2))
    turnover = np.sum(np.abs(beta[1:] - beta[:-1])) / len(beta)
    return tracking_error, turnover

In [43]:
def rolling_window_backtest(stock_returns, stock_index, train_window, test_window, kact, lambdas):
    results = {'CARD': [], 'ALASSO': [], 'SLOPE-SLC': [], 'MSW-LASSO': []}
    n = len(stock_returns)
    
    for start in range(0, n - train_window - test_window + 1, test_window):
        train_data = stock_returns.iloc[start:start + train_window]
        test_data = stock_returns.iloc[start + train_window:start + train_window + test_window]
    
        # Filter out columns with NaN values in the current window
        valid_assets_train = train_data.dropna(axis=1, how='any')
        valid_assets_test = test_data.dropna(axis=1, how='any')
        valid_assets = valid_assets_train.columns.intersection(valid_assets_test.columns)
        
        X_train = valid_assets_train[valid_assets].values
        y_train = stock_index.iloc[start:start + train_window].values
        
        X_test = valid_assets_test[valid_assets].values
        y_test = stock_index.iloc[start + train_window:start + train_window + test_window].values
        
        # Skip if no valid assets are available
        if X_train.shape[1] == 0 or X_test.shape[1] == 0:
            continue
        
        # CARD
        beta_card = card(X_train, y_train, kact)
        te_card, to_card = evaluate_portfolio(X_test, y_test, beta_card)
        results['CARD'].append({'date': stock_returns.index[start + train_window], 'te': te_card, 'to': to_card})
        
        # ALASSO
        beta_alasso = alasso(X_train, y_train, kact)
        te_alasso, to_alasso = evaluate_portfolio(X_test, y_test, beta_alasso)
        results['ALASSO'].append({'date': stock_returns.index[start + train_window], 'te': te_alasso, 'to': to_alasso})
        
        # SLOPE-SLC
        beta_slope_slc = slope_slc(X_train, y_train, kact, lambdas)
        te_slope_slc, to_slope_slc = evaluate_portfolio(X_test, y_test, beta_slope_slc)
        results['SLOPE-SLC'].append({'date': stock_returns.index[start + train_window], 'te': te_slope_slc, 'to': to_slope_slc})
        
        # MSW-LASSO
        beta_msw_lasso = msw_lasso(X_train, y_train, kact)
        te_msw_lasso, to_msw_lasso = evaluate_portfolio(X_test, y_test, beta_msw_lasso)
        results['MSW-LASSO'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso, 'to': to_msw_lasso})
        
        print(f"Window ending {stock_returns.index[start + train_window].date()}:")
        print(f"CARD: TE={te_card:.4f}, TO={to_card:.4f}")
        print(f"ALASSO: TE={te_alasso:.4f}, TO={to_alasso:.4f}")
        print(f"SLOPE-SLC: TE={te_slope_slc:.4f}, TO={to_slope_slc:.4f}")
        print(f"MSW-LASSO: TE={te_msw_lasso:.4f}, TO={to_msw_lasso:.4f}")
    
    return results


In [44]:
# Set parameters
kact = 50
lambdas = np.linspace(0.1, 1, stock_returns.shape[1]-1)  # Adjust as needed
train_window = 250
test_window =21

# DATA usage
X = stock_returns.drop(columns=['^GSPC']).values
y = stock_returns['^GSPC'].values



kact = 50  # Number of active stocks
beta = msw_lasso(X, y, kact)

# Perform the rolling window backtest
results = rolling_window_backtest(stock_returns, stock_data['kospi_index'], train_window, test_window, kact, lambdas)

ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


SolverError: Workspace allocation error!

### 시각화

In [ ]:


# Convert results to DataFrame
df_results = {key: pd.DataFrame(val).set_index('date') for key, val in results.items()}

# Plot the results
plt.figure(figsize=(14, 7))

for key in df_results:
    plt.plot(df_results[key]['te'], label=f'{key} Tracking Error')

plt.title('Tracking Error Over Time')
plt.xlabel('Date')
plt.ylabel('Tracking Error')
plt.legend()
plt.show()

plt.figure(figsize=(14, 7))

for key in df_results:
    plt.plot(df_results[key]['to'], label=f'{key} Turnover')

plt.title('Turnover Over Time')
plt.xlabel('Date')
plt.ylabel('Turnover')
plt.legend()
plt.show()
